In [1]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, GlobalAveragePooling2D, Dropout, Flatten
from tensorflow.keras import models, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

2021-12-01 15:21:25.907646: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-01 15:21:25.907739: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
resnet50_128_base = ResNet50(
    include_top=False, weights='imagenet',
    input_shape=(128,128,3),classes=2
)

for layer in resnet50_128_base.layers:
    layer.trainable = False

resnet50_128 = models.Sequential()
resnet50_128.add(resnet50_128_base)
resnet50_128.add(GlobalAveragePooling2D())
resnet50_128.add(Dense(1024,activation='relu'))
resnet50_128.add(Dropout(0.5))
resnet50_128.add(Dense(1,activation='sigmoid'))

resnet50_128.compile(optimizer=Adam(learning_rate=0.0001),loss='binary_crossentropy',metrics=['accuracy','Recall'])

2021-12-01 15:21:28.620156: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: UNKNOWN ERROR (100)
2021-12-01 15:21:28.620272: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (LAPTOP-DUHMQ01S): /proc/driver/nvidia/version does not exist
2021-12-01 15:21:28.620688: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
resnet50_128_base.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 134, 134, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 64, 64, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 64, 64, 64)   256         ['conv1_conv[0][0]']      

In [4]:
#for layer in resnet50_128_base.layers[:171]:
    #layer.trainable = False
    
#for layer in resnet50_128_base.layers[171:]:
    #layer.trainable = True
    
for i, layer in enumerate(resnet50_128_base.layers):
    print(i, layer.name, layer.trainable)

0 input_1 False
1 conv1_pad False
2 conv1_conv False
3 conv1_bn False
4 conv1_relu False
5 pool1_pad False
6 pool1_pool False
7 conv2_block1_1_conv False
8 conv2_block1_1_bn False
9 conv2_block1_1_relu False
10 conv2_block1_2_conv False
11 conv2_block1_2_bn False
12 conv2_block1_2_relu False
13 conv2_block1_0_conv False
14 conv2_block1_3_conv False
15 conv2_block1_0_bn False
16 conv2_block1_3_bn False
17 conv2_block1_add False
18 conv2_block1_out False
19 conv2_block2_1_conv False
20 conv2_block2_1_bn False
21 conv2_block2_1_relu False
22 conv2_block2_2_conv False
23 conv2_block2_2_bn False
24 conv2_block2_2_relu False
25 conv2_block2_3_conv False
26 conv2_block2_3_bn False
27 conv2_block2_add False
28 conv2_block2_out False
29 conv2_block3_1_conv False
30 conv2_block3_1_bn False
31 conv2_block3_1_relu False
32 conv2_block3_2_conv False
33 conv2_block3_2_bn False
34 conv2_block3_2_relu False
35 conv2_block3_3_conv False
36 conv2_block3_3_bn False
37 conv2_block3_add False
38 conv2_bloc

In [5]:
resnet50_128.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 4, 4, 2048)        23587712  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 1024)              2098176   
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                                 
 dense_1 (Dense)             (None, 1)                 1025      
                                                                 
Total params: 25,686,913
Trainable params: 2,099,201
Non-trainable params: 23,587,712
____________________________________

In [13]:
from tensorflow.keras.preprocessing import image_dataset_from_directory
data_train = image_dataset_from_directory('../raw_data/malaria/',
                                        labels ='inferred',
                                        label_mode='binary',
                                        color_mode='rgb',
                                        batch_size=128,
                                        image_size=(128,128),
                                        shuffle=True,
                                        seed=42,
                                        validation_split=0.2,
                                        subset='training',
                                        crop_to_aspect_ratio=False
                                         )

Found 27558 files belonging to 2 classes.
Using 22047 files for training.


In [14]:
data_val = image_dataset_from_directory('../raw_data/malaria/',
                                        labels ='inferred',
                                        label_mode='binary',
                                        color_mode='rgb',
                                        batch_size=128,
                                        image_size=(128,128),
                                        shuffle=True,
                                        seed=42,
                                        validation_split=0.2,
                                        subset='validation'
                                       )

Found 27558 files belonging to 2 classes.
Using 5511 files for validation.


In [15]:
es = EarlyStopping(patience=5,restore_best_weights=True)

In [16]:
history = resnet50_128.fit(data_train, validation_data=data_val, callbacks=[es], epochs=100)

Epoch 1/100
173/173 [==============================] - 509s 3s/step - loss: 0.3147 - accuracy: 0.8684 - recall: 0.8715 - val_loss: 0.1927 - val_accuracy: 0.9242 - val_recall: 0.9374
Epoch 2/100
173/173 [==============================] - 513s 3s/step - loss: 0.2155 - accuracy: 0.9116 - recall: 0.9151 - val_loss: 0.1763 - val_accuracy: 0.9318 - val_recall: 0.9252
Epoch 3/100
173/173 [==============================] - 512s 3s/step - loss: 0.1939 - accuracy: 0.9241 - recall: 0.9302 - val_loss: 0.1754 - val_accuracy: 0.9300 - val_recall: 0.9617
Epoch 4/100
173/173 [==============================] - 512s 3s/step - loss: 0.1817 - accuracy: 0.9297 - recall: 0.9338 - val_loss: 0.1654 - val_accuracy: 0.9361 - val_recall: 0.9238
Epoch 5/100
173/173 [==============================] - 519s 3s/step - loss: 0.1743 - accuracy: 0.9322 - recall: 0.9362 - val_loss: 0.1601 - val_accuracy: 0.9367 - val_recall: 0.9606
Epoch 6/100
 33/173 [====>.........................] - ETA: 7:26 - loss: 0.1627 - accuracy

KeyboardInterrupt: 